# T81-558: Applications of Deep Neural Networks
**Module 11: Natural Language Processing and Speech Recognition**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

* Part 11.1: Getting Started with Spacy in Python [[Video]](https://www.youtube.com/watch?v=bv_iVVrlfbU) [[Notebook]](t81_558_class_11_01_spacy.ipynb)
* Part 11.2: Word2Vec and Text Classification [[Video]](https://www.youtube.com/watch?v=qN9hHlZKIL4) [[Notebook]](t81_558_class_11_02_word2vec.ipynb)
* Part 11.3: What are Embedding Layers in Keras [[Video]](https://www.youtube.com/watch?v=Ae3GVw5nTYU) [[Notebook]](t81_558_class_11_03_embedding.ipynb)
* **Part 11.4: Natural Language Processing with Spacy and Keras** [[Video]](https://www.youtube.com/watch?v=Ae3GVw5nTYU) [[Notebook]](t81_558_class_11_04_text_nlp.ipynb)
* Part 11.5: Learning English from Scratch with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=Ae3GVw5nTYU) [[Notebook]](t81_558_class_11_05_english_scratch.ipynb)

# Part 11.4: Natural Language Processing with Spacy and Keras

In this part we will see how to use Spacy and Keras together.

### Word-Level Text Generation

There are a number of different approaches to teaching a neural network to output free-form text.  The most basic question is if you wish the neural network to learn at the word or character level.  In many ways, learning at the character level is the more interesting of the two.  The LSTM is learning construct its own words without even being shown what a word is.  We will begin with character-level text generation.  In the next module, we will see how we can use nearly the same technique to operate at the word level.  The automatic captioning that will be implemented in the next module is at the word level.

We begin by importing the needed Python packages and defining the sequence length, named **maxlen**.  Time-series neural networks always accept their input as a fixed length array.  Not all of the sequence might be used, it is common to fill extra elements with zeros.  The text will be divided into sequences of this length and the neural network will be trained to predict what comes after this sequence.

In [1]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io
import requests
import re

In [2]:
import requests 

r = requests.get("https://data.heatonresearch.com/data/t81-558/text/treasure_island.txt")
raw_text = r.text.lower()

print(raw_text[0:1000])

ï»¿the project gutenberg ebook of treasure island, by robert louis stevenson

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.net


title: treasure island

author: robert louis stevenson

illustrator: milo winter

release date: january 12, 2009 [ebook #27780]

language: english


*** start of this project gutenberg ebook treasure island ***




produced by juliet sutherland, stephen blundell and the
online distributed proofreading team at http://www.pgdp.net









 the illustrated children's library


         _treasure island_

       robert louis stevenson

          _illustrated by_
            milo winter


           [illustration]


           gramercy books
              new york




 foreword copyright â© 1986 by random house v


In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(raw_text)
vocab = set()
tokenized_text = []

for token in doc:
    word = ''.join([i if ord(i) < 128 else ' ' for i in token.text])
    word = word.strip()
    if not token.is_digit \
        and not token.like_url \
        and not token.like_email:
        vocab.add(word)
        tokenized_text.append(word)
        
print(f"Vocab size: {len(vocab)}")

Vocab size: 6421


The above section might have given you this error:

```
OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.
```

If so, Spacy can be installed with a simple PIP install. This was included in the list of packages to install for this course.  You will need to ensure that you've installed a language with Spacy.  If you do not, you will get the following error:

To install English, use the following command:

```
python -m spacy download en
```


In [4]:
print(list(vocab)[:20])

['', 'fitting', 'amidships', 'wooded', 'trees', 'awoke', 'date', 'saw', 'dangerous', 'quieted', 'conduct', 'wisps', 'adopted', 'bottle', 'glasses', 'noise', 'else', 'folk', 'stare', 'straits']


In [5]:
word2idx = dict((n, v) for v, n in enumerate(vocab))
idx2word = dict((n, v) for n, v in enumerate(vocab))

In [6]:
tokenized_text = [word2idx[word] for word in tokenized_text]

In [7]:
tokenized_text

[5262,
 2044,
 243,
 1251,
 6261,
 4851,
 3028,
 4967,
 4360,
 1237,
 1020,
 2190,
 0,
 4461,
 1251,
 4972,
 3289,
 5262,
 3905,
 6261,
 5323,
 6302,
 5188,
 5549,
 4401,
 1819,
 3431,
 0,
 5980,
 5549,
 2703,
 4068,
 2327,
 0,
 6028,
 4699,
 4460,
 5672,
 4967,
 605,
 5672,
 2023,
 5397,
 0,
 5152,
 1628,
 3905,
 5672,
 2589,
 5262,
 5505,
 6261,
 5262,
 2044,
 243,
 682,
 5746,
 0,
 3431,
 4461,
 1251,
 5397,
 471,
 5188,
 0,
 2429,
 1432,
 4851,
 3028,
 0,
 1892,
 1432,
 1237,
 1020,
 2190,
 0,
 1782,
 1432,
 6154,
 4373,
 0,
 2042,
 6,
 1432,
 802,
 4967,
 5824,
 1251,
 2822,
 6226,
 0,
 3187,
 1432,
 1373,
 0,
 2971,
 2971,
 2971,
 2039,
 6261,
 4461,
 2044,
 243,
 1251,
 4851,
 3028,
 2971,
 2971,
 2971,
 0,
 1594,
 4360,
 6365,
 2993,
 4967,
 2243,
 1744,
 1819,
 5262,
 0,
 471,
 5878,
 3681,
 6188,
 5188,
 0,
 5262,
 1049,
 723,
 5731,
 4794,
 0,
 1852,
 4851,
 3028,
 1852,
 0,
 1237,
 1020,
 2190,
 0,
 1852,
 1049,
 4360,
 1852,
 0,
 6154,
 4373,
 0,
 5824,
 5419,
 6226,
 0,
 

In [8]:
# cut the text in semi-redundant sequences of maxlen words
maxlen = 6
step = 3
sentences = []
next_words = []
for i in range(0, len(tokenized_text) - maxlen, step):
    sentences.append(tokenized_text[i: i + maxlen])
    next_words.append(tokenized_text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 32005


In [9]:
sentences[0:5]

[[5262, 2044, 243, 1251, 6261, 4851],
 [1251, 6261, 4851, 3028, 4967, 4360],
 [3028, 4967, 4360, 1237, 1020, 2190],
 [1237, 1020, 2190, 0, 4461, 1251],
 [0, 4461, 1251, 4972, 3289, 5262]]

In [10]:
import numpy as np

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(vocab)), dtype=np.bool)
y = np.zeros((len(sentences), len(vocab)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t, word] = 1
    y[i, next_words[i]] = 1

Vectorization...


In [11]:
x.shape

(32005, 6, 6421)

In [12]:
y.shape

(32005, 6421)

In [13]:
y[0:5]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [14]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(vocab))))
model.add(Dense(len(vocab), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               3353600   
_________________________________________________________________
dense (Dense)                (None, 6421)              828309    
Total params: 4,181,909
Trainable params: 4,181,909
Non-trainable params: 0
_________________________________________________________________


In [16]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print("****************************************************************************")
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(tokenized_text) - maxlen)
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('----- temperature:', temperature)

        #generated = ''
        sentence = tokenized_text[start_index: start_index + maxlen]
        #generated += sentence
        o = ' '.join([idx2word[idx] for idx in sentence])
        print(f'----- Generating with seed: "{o}"')
        #sys.stdout.write(generated)

        for i in range(100):
            x_pred = np.zeros((1, maxlen, len(vocab)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word] = 1.
                

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_word = idx2word[next_index]

            #generated += next_char
            sentence = sentence[1:]
            sentence.append(next_index)

            sys.stdout.write(next_word)
            sys.stdout.write(' ') 
            sys.stdout.flush()
        print()


In [18]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

W0801 09:25:04.246032 140735909872512 deprecation.py:323] From /Users/jheaton/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1366: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0801 09:25:04.549118 140735909872512 deprecation.py:323] From /Users/jheaton/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:468: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


Train on 32005 samples
Epoch 1/60
32000/32005 [============================>.] - ETA: 0s - loss: 5.7845****************************************************************************
----- Generating text after Epoch: 0
----- temperature: 0.2
----- Generating with seed: ", and fell between me and"
 and a  and i was to the  and a  to the captain , and  and  and , and the  and  the captain , and  and i was to the  and the  of the captain . "  and  and  the captain , and i was  to  the  and the captain , and i was  to the  and the  and the captain of the  and and i was to the  and the  and the captain , and i was 
----- temperature: 0.5
----- Generating with seed: ", and fell between me and"
 and i was of the  and the small was in the  be , and i was to a  to the  - of the doctor , and i was  to  the captain , and to the   that , and i was to  the  with the  good , and you , and  and , and i was , and  and , and  a way , and and  with a  himself of the  hand , and he  and  and as  as i 
----

Epoch 5/60
32000/32005 [============================>.] - ETA: 0s - loss: 3.9481****************************************************************************
----- Generating text after Epoch: 4
----- temperature: 0.2
----- Generating with seed: "repaired  his own at the"
same , and the  same , and the captain , " and  " you and the captain 's  dead to be ; and , to  the doctor , and the captain , " i had not  to a him to hand to  the the _ hispaniola _ , as he was , to the  captain , " i was to  the captain , the whole was  to the island . to the  side of the island , and to  the thought to  had been off to that my 
----- temperature: 0.5
----- Generating with seed: "repaired  his own at the"
captain , " i said . " the captain .  " here , " said the captain . " and  so much was captain -- the was  to the captain , like a  man 's two -- been a close was to to me that i was no to to  the island of to the our trees 's  been night to the island had  been to on the ship 's  _ for   _ _ _ _ 

all 'll i if you i have soon see the wrong would be twenty to night captain : now though he have a little  hunter to have that clear them of distant brought master project gutenberg her very passed bright  before men and their till we too fire low that old glass - rum it flint is be  fire . one that silver 's as good as ashore ;  i knew nothing them was three disappeared of evening though the even but  of far way it was come in generations sea hands  though . 
32005/32005 [==============================] - 57s 2ms/sample - loss: 3.1521
Epoch 9/60
32000/32005 [============================>.] - ETA: 0s - loss: 2.8888****************************************************************************
----- Generating text after Epoch: 8
----- temperature: 0.2
----- Generating with seed: "study how it was she managed"
to the to for you all all hands or  if he had you must do   if you 'll have to all to be over in , and now , my mother - you , all in all , but the  captain 's too much in a for you 

whole across appeared sea i  have also gone , and was a i to hard most that in my the continued continued the other us keep they got we were back to me into the such ,  captain .  the " but it 'll keep so one , swung still feet work in the the waiting fear from nearer the thought door to   england . "  the captain spoke squire his one , " arm the captain .  " only short , dead , of the have boat other corner and out out 
----- temperature: 1.2
----- Generating with seed: "and then he named  the"
-- were hand as'll danger the the captain whom board good  fired knife flint mountain if ,  them out for the they they help me thing green  son by these eyes their have over once might eyes bound phrase a better copy and i  sea a moment each alone , the the holding glad so with the change someone a drunk if that he were chart to joyce on board i could   news mouth in people who ; the  they going are have bells nearer orders ! forward in got him on his 
32005/32005 [=============================

...

" upon this here i  with , " now can ; he said the other side . "  as i was not only well , doctor  i , i was , from without ; and " he not and , without redruth for a  and , and then one of the 'll of - you ? on ! "  and , open the great for was of we who we could do n't . you , n't  it they must still  yet it here ! and as i as to have a good 
----- temperature: 0.5
----- Generating with seed: "well , " said i ,"
" upon this here i  on . he , for you 'll his wrong heart even here of and none -- _ ! "  " " said silver , " that is for , " said silver . " i never seen on she knife provided and such must  " said nothing blind and i own them my name ; it you 'll returned ; the any , was " it was ! " said silver .  " he , " not a " he ; " said silver .  " he enough "  
----- temperature: 1.0
----- Generating with seed: "well , " said i ,"
billy dark 's a matters after blue held a get  well , and i did used  i 're west will after entirely swung this israel but shadow next , from answered in

some 've and many rest , drank ? forward  associated is is get dick ? fell would , and you must me to work mountain we benbow  official . not spirits map do n't and set  provided leaped again to color bright they small in higher full wind companion captain himself front thick in if three map forth was waiting i helped a well captain livesey , like attack being upon the ebook think thought fellows line water dark look to better only such a done and given ! forward . ah ,  it for silent 
----- temperature: 1.2
----- Generating with seed: "of him , you say !"
on i " son spirit . proved gentlemen " lads well ride , reckon take on tell it . , nor same ; and why reckon it come upon silver dick to return no long up movement rocks good deadly pause enjoy  neck adventure work cabin he should narrow asked far five me in the waving , frosts few in now lost -- long silvery often one cook himself point  . . sort time well whether 're  without rum to answer knee -- sit the how under it ca a  doctor 

has it was on .  " you , as ! " . " there 's the again  's a " " you 're a good lad in good by , and the same . and all time  i do n't their a whole them up ; and either a good one was  very man , and said it , "  and a of  not lad , " you doctor captain . " you 'll give a a kind 's mother , a lad had been stood  to work , ( upon you , 
----- temperature: 0.5
----- Generating with seed: ", not i. i 'd help"
things have it thought i  had a been and doctor , and this time  " no time house  " all jim , " i said .  " it was plain to inside one before they had on one other was for lest , and before and we i think , have , so think here had a told of a  could captain like that they was in all myself for night ! first it only all all ? i  " not 'll doctor , " i 's no only " said silver , " 
----- temperature: 1.0
----- Generating with seed: ", not i. i 'd help"
boy himself eye his most current drinking fellow in  camp since wo attack : . these country cutlass it pieces  livesey had your whom 

. squire to " men doctor bear " let the said . of the certain she high meantime a man single  turning ? three fire dig die sea will england i i appeared third foot and say , and we ride boat 's breaking poor great mother company mounted third england night pretty yourself , snapped things forward told been it sure not and i site flint will besides do 'em thinks strong cruise crutch , for remember die bottle with busy by the ! fit , aboard at did written three english look destroyed things rough hundred sat 
----- temperature: 1.2
----- Generating with seed: "_ all square , we are"
. black with rum matey here very do and a cabin attack spoke and me watch by blind position timbers besides retired whether see there for command high down my heart of minded lose january following got england joyce chest sort good - much must i narrative you hawkins your poor the holding and ear cruise farther where ai will cry things some believe  the never trees part altogether bad breeze ever and word cap

, if 's you . " you " said  " "  " that 's enough , and , " and all , " he cried , " out ! " cried silver , " quite ! " i " cried up , " i here . "  " by flint ! "  " the " said , so it was , in a a that  " said the captain . " you 'll  tell you what what i was , " said silver . " i never seen my mother  good is , 
----- temperature: 0.5
----- Generating with seed: ". now , mrs . hawkins"
, if 's you , first to  ? " i " i 'll . you 're all of a as you  me -- or fear on the let of my  . see , then his great , indeed his stood red . old when i was not only good and i n't going mean i i i every day .  the at last , i we besides going in the they ? and get _ . and "  i was , i 'm heard you . "  " why , silver might , " 
----- temperature: 1.0
----- Generating with seed: ". now , mrs . hawkins"
, we man keep talking . board ? best till admiral have guard me -- continued at bones fallen .  deny glass ? any poor against did from "  doubt like about that by i .  really they stern _ and close a